# <font color=blue>**TAREA 2**

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from numpy import genfromtxt
from scipy.stats import multivariate_normal
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

## <font color=blue>**Parte 2.1 - Deteccion de Anomalias**

- Usando el dataset de estaturas(en google drive) aplicar detección de anomalías
    - Usar como training-set la primera pestaña “normales” (y=0)
    - La segunda pestaña “valtest(normales)” contiene datos normales(y=0):
        - Usar la mitad para cross-validation
        - Usar la mitad para testing
    - La tercera pestaña “valtest(anomalias)” contiene anomalías(y=1):
        - Usar la mitad para cross-validation 
        - Usar la mitad para testing
- Los datos de cross-validation deben ser usados para selección de hyper-parámetros(por ejemplo el umbral epsilon) y/o selección de transformaciones a aplicar a las variables.
- Usar los datos de testing para reportar las métricas de evaluación apropiadas
- Es permitido usar librerías estadísticas(por ejemplo scipy.stats) para cosas como:
    - Estimación de parámetros(función de densidad)
    - Cálculo de densidades y/o probabilidades con la función de densidad estimada


In [2]:
EstaturasN = pd.read_excel('estaturas.xlsx','normales',dtype={'statura(metros)': float, 'Edad(años)': float})
EstaturasVT = pd.read_excel('estaturas.xlsx','valtest(normales)',dtype={'statura(metros)': float, 'Edad(años)': float})
EstaturasVA = pd.read_excel('estaturas.xlsx','valtest(anomalias)',dtype={'Estatura': float, 'Edad': float})
EstaturasVA

,Estatura,Edad
0,0.25,2.4
1,175.10,2.5
2,0.15,250.0
3,150.00,14.0


In [3]:
#SPLIT DE DATOS
VT_Cross, VT_Test = train_test_split(EstaturasVT, test_size=0.5)
VA_Cross, VA_Test = train_test_split(EstaturasVA, test_size=0.5)

EstaturasC = np.vstack((np.array(VT_Cross),np.array(VA_Cross)))
EstaturasT = np.vstack((np.array(VT_Test),np.array(VA_Test)))
EstaturasL = np.concatenate((np.zeros(VT_Cross.shape[0]),np.ones(VA_Cross.shape[0])))

In [4]:
def Calculo_MeanSD(Data):
    Mean = np.mean(Data, axis=0)
    StandardD = np.cov(Data.T)    
    return Mean, StandardD

In [5]:
def Calculo_Densidad(Data,Mean, StandardD):
    return multivariate_normal(mean =Mean, cov= StandardD ).pdf(Data)

In [6]:
def DeteccionAnomalia_Training(Data_Train, Data_Cross, Data_Label):
    
    m1, sd1 = Calculo_MeanSD(Data_Train)
    prob = Calculo_Densidad(Data_Train, m1,sd1)
    
    prob_a = Calculo_Densidad(Data_Cross, m1, sd1)
    
    best_epsilon = 0
    best_f1 = 0
    f = 0
    stepsize = (max(prob_a) - min(prob_a)) / 1000
    epsilons = np.arange(min(prob_a),max(prob_a),stepsize)
    for epsilon in np.nditer(epsilons):
        predictions = (prob_a < epsilon) 
        f = f1_score(Data_Label, predictions,average='binary')
        if f > best_f1:
            best_f1 = f
            best_epsilon = epsilon
    
    return best_f1, best_epsilon, m1, sd1


In [7]:
f1, epsilon, m, sd = DeteccionAnomalia_Training(EstaturasN, EstaturasC, EstaturasL)

In [8]:
def DeteccionAnomalia_Prediction(Data_Test, epsilon, m, sd):
    prob = Calculo_Densidad(Data_Test, m, sd)
            
    return prob<epsilon

In [9]:
DeteccionAnomalia = DeteccionAnomalia_Prediction(EstaturasT, epsilon, m, sd)
print(DeteccionAnomalia)

[False False False False  True  True]


In [10]:
print("Anomalias")
print(EstaturasT[DeteccionAnomalia])

Anomalias
[[1.5e-01 2.5e+02]
 [2.5e-01 2.4e+00]]


## <font color=blue>**Parte 2.2 - Reduccion de Dimensionalidad**

Usando el dataset de fashion MNIST realizar:
- PCA sin sklearn para reducir a 2 dimensiones.
    - Anotar la cantidad de varianza preservada.
- t-sne con sklearn para reducir a 2 dimensiones.
- Analizar ambas representaciones, comparar y concluir.
- Aplicar clustering con sklearn sobre la representación reducida. 
- Ya que este dataset si posee etiquetas “y” (tipo de prenda) analizar si los clusters encontrados tienden a agrupar el mismo tipo de prenda o prendas similares(por ejemplo se puede graficar cada tipo de diferente color)
- Agregar conclusiones finales. 

In [ ]:
from tensorflow import keras

In [ ]:
FMist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = FMist.load_data()

In [ ]:
np.unique(train_labels)

In [ ]:
train_images.shape

In [ ]:
# reshape dataset 
n, v1, v2 = train_images.shape
train_imagesRS = train_images.reshape((n,v1*v2))
train_imagesRS.shape

**PCA**

**T-SNE**

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
%%time
X_TSNE =  TSNE(n_components=2)
Z = X_TSNE.fit_transform(train_imagesRS)
#print(X_TSNE.shape)

**KMeans**

In [ ]:
from sklearn.cluster import KMeans

kmeans1 = KMeans(n_clusters=5, random_state=0).fit(Z)
print(kmeans1.labels_)
print(kmeans1.cluster_centers_)